In [2]:
from multiprocessing.dummy import Pool
from PIL import Image, ImageDraw
import numpy as np
import cv2
from imutils import paths

ORIG_INPUT_DATASET = "datasets/orig"



def calculate_mean(filename, position, no_dot):
    # load image
    img = Image.open(filename).convert('LA')
    img_cropped = img.crop(position)
    w, h = img_cropped.size
    mask = Image.new('L', img_cropped.size)
    mask_draw = ImageDraw.Draw(mask)
    width, height = img_cropped.size
    mask_draw.ellipse((0, 0, width, height), fill=255)
    img_cropped.putalpha(mask)

    img_cropped.convert('RGBA')
    data = np.array(img_cropped)
    black_pixels = np.where((data[:, :, 0] == 0) | (data[:, :, 1] == 0))

    # set those pixels to white
    data[black_pixels] = [255]

    for item in data:
        for part in item:
            if part[0] == 255:
                part[1] = 0

    # Image.fromarray(data).save('dot' + str(no_dot) + '.png', "PNG")

    mean = np.mean(data)
    return mean

def average(list):
    return sum(list)/len(list)

imagePaths = list(paths.list_images(ORIG_INPUT_DATASET))
# first_dot_postion = (289, 193, 313, 217)
# second_dot_postion = (380, 189, 404, 213)
first_dot_postion = (294, 198, 309, 213)
second_dot_postion = (384, 194, 399, 209)

class0 = []
class1 = []

for image_path in imagePaths:
    first_dot_mean = calculate_mean(image_path, first_dot_postion, 1)
    second_dot_mean = calculate_mean(image_path, second_dot_postion, 2)
    ratio = first_dot_mean / second_dot_mean
    
    if image_path[-5:-4] == "1":
        class1.append(ratio)
    else:
        class0.append(ratio)

print(len(class0))
print(len(class1))
#print(imagePaths)

print("average: ", average(class0))
print("min: ", min(class0))
print("max: ", max(class0))
print()
print("average: ", average(class1))
print("min: ", min(class1))
print("max: ", max(class1))

class01 = class0 + class1
print(len(class01))




print()
class12 = list(zip(class0, [0] * len(class0)))
class12 += list(zip(class1, [1] * len(class1)))
print(len(class12))

results = []

def calc_acc(threshold):
    global results


    class0_test = []
    class1_test = []
    for x in class12:
        if x[0] < threshold:
            class0_test.append(x)
        else:
            class1_test.append(x)

    good1 = 0
    for x in class0_test:
        if x[1] == 0:
            good1 += 1
    acc1 = good1/len(class0)

    good2 = 0
    for x in class1_test:
        if x[1] == 1:
            good2 += 1
    acc2 = good2/len(class1)

    results.append((acc1, acc2, abs(acc1 + acc2), threshold))
    confusion_matrix = [
        [good2, len(class1_test) - good2], [len(class0_test) - good1, good1]
        ]
    return confusion_matrix, acc1, acc2





for x in [i for i in np.arange(1, 1.13, 0.005)]:
    calc_acc(x)



print(sorted(results,reverse=True, key=lambda x : x[2]))
best_case = sorted(results,reverse=True, key=lambda x : x[2])[0]


print(best_case)
print()


mat, acc1, acc2 = calc_acc(best_case[-1])
print(mat[0])
print(mat[1])
print(acc1, acc2)


170
380
average:  1.0504269039619671
min:  0.965046832245843
max:  1.132986408202769

average:  1.1119375324887144
min:  0.992600764930862
max:  1.225154600537198
550

550
[(0.7823529411764706, 0.7894736842105263, 1.571826625386997, 1.0749999999999984), (0.7529411764705882, 0.8131578947368421, 1.5660990712074303, 1.0699999999999985), (0.8, 0.7421052631578947, 1.5421052631578949, 1.0799999999999983), (0.6941176470588235, 0.8421052631578947, 1.5362229102167182, 1.0649999999999986), (0.6352941176470588, 0.8763157894736842, 1.511609907120743, 1.0599999999999987), (0.8058823529411765, 0.7026315789473684, 1.5085139318885448, 1.0849999999999982), (0.8764705882352941, 0.6157894736842106, 1.4922600619195046, 1.094999999999998), (0.5882352941176471, 0.9, 1.488235294117647, 1.0549999999999988), (0.8941176470588236, 0.5815789473684211, 1.4756965944272447, 1.0999999999999979), (0.8235294117647058, 0.6473684210526316, 1.4708978328173374, 1.089999999999998), (0.9117647058823529, 0.5342105263157895, 1